# <h1 align="center" style="font-size:200%; color:blue">Microsoft’s entrance to Movie business</h1><br>
<h1 align="center" style="font-size:200%; color:blue">Data loading and cleaning and exploration</h1> <br>

<p style="color:black"> First I shall explore and clean the data files provided for the project. Following table contains the 11 files categorized by the data sources. </p> <br>

|      __IMDB__                |    __Box Office Mojo__   |   __Rotten Tomatoes__  | __TheMovieDB.org__  |
|------------------------------|--------------------------|------------------------|---------------------|
| 1. imdb.name.basics.csv      | 7. bom.movie_gross.csv   | 8. rt.movie_info.tsv   | 10. tmdb.movies.csv |
| 2. imdb.title.akas.csv       |                          | 9. rt.reviews.tsv      |                     |
| 3. imdb.title.basics.csv     |                          |                        |                     |
| 4. imdb.title.crew.csv       |                          |                        |                     |
| 5. imdb.title.principals.csv |                          |                        |                     |
| 6. imdb.title.ratings.csv    |                          |                        |                     |
|------------------------------|--------------------------|------------------------|---------------------|
|11. tn.movie_budgets.csv      |                          |                        |                     |

<p style="font-size:100%; color:green"> Methodology:  <b>R</b>OSEMED</p>
<!--><p style="color:black"> <b>R</b>OSEMED method </p> <-->

In [2]:
#importing libraries
import pandas as pd 
import numpy as np

# Data loading and cleaning

## cleaning the-numbers.com data file

In [3]:
# original source: https://www.the-numbers.com/movie/budgets
tn_budgets = pd.read_csv('Data/tn.movie_budgets.csv')
display(tn_budgets.head(4), len(tn_budgets))

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"


5782

In [4]:
# id column useless 
tn_budgets = tn_budgets.drop('id', axis = 1)
# format date
tn_budgets['release_date'] = pd.to_datetime(tn_budgets['release_date'])
# clean movie column
tn_budgets.movie = tn_budgets.movie.str.strip() 
# clean movie domestic_gross, production_budget and worldwide_gross
tn_budgets['domestic_gross'] = tn_budgets['domestic_gross'].str.replace('$', '')
tn_budgets['domestic_gross'] = tn_budgets['domestic_gross'].str.replace(',', '')
tn_budgets['domestic_gross'] = tn_budgets['domestic_gross'].astype(float)
tn_budgets['production_budget'] = tn_budgets['production_budget'].str.replace('$', '')
tn_budgets['production_budget'] = tn_budgets['production_budget'].str.replace(',', '')
tn_budgets['production_budget'] = tn_budgets['production_budget'].astype(float)
tn_budgets['worldwide_gross'] = tn_budgets['worldwide_gross'].str.replace('$', '')
tn_budgets['worldwide_gross'] = tn_budgets['worldwide_gross'].str.replace(',', '')
tn_budgets['worldwide_gross'] = tn_budgets['worldwide_gross'].astype(float)
# tn_budgets.isna().sum()  # checked it's clean

## cleaning Box Office Mojo data file

In [5]:
# Box Office Mojo dataset
bom = pd.read_csv('Data/bom.movie_gross.csv')
display(bom.head(2),len(bom))

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


3387

In [6]:
#bom.info()   # 3387 entries
# The Null values in domestic_gross and foreign_gross are tragged "-1.0 float64"
# and studio with 'unknown'
bom['studio'].fillna(value='unknow',inplace=True)
bom['domestic_gross'] = pd.to_numeric(bom['domestic_gross'],errors='coerce')
bom['domestic_gross'].fillna(value=-1.0,inplace=True)
bom['foreign_gross'] = pd.to_numeric(bom['foreign_gross'],errors='coerce')
bom['foreign_gross'].fillna(value=str(-1.0),inplace=True)
bom = bom[bom.foreign_gross != -1.0]
#bom.isna().sum() # it's clean now

## cleaning Rotten Tomatoes data files

In [7]:
# Rotten Tomatoes
rt = pd.read_csv('Data/rt.movie_info.tsv', sep='\t')
display(rt.head(2),len(rt))

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One


1560

In [8]:
# we'll drop ['synopsis', 'genre', 'box_office']
rt.drop(columns=['synopsis', 'genre', 'box_office', 
                'director', 'writer', 'dvd_date', 'currency', 'box_office', 'studio'], inplace=True) # has much more 
# comprihencive data in IMDB
rt['rating'] = pd.to_numeric(rt['rating'],errors='coerce')
rt['rating'].fillna(value=-1.0,inplace=True)
rt = rt[rt.rating != -1.0]
rt['runtime']=rt.runtime.map(lambda x: x.strip().replace("minutes",""), na_action='ignore')
rt['runtime'] = pd.to_numeric(rt['runtime'],errors='coerce')
rt['runtime'].fillna(value=-1.0,inplace=True)
rt['theater_date'].fillna(value='unknown',inplace=True)
#rt.isna().sum() # It's clean now

Series([], Name: theater_date, dtype: object)

In [9]:
# review's file maybe useful for reviews and ratings. 
# encoding worked!!! with correct delimiter
rt_reviews = pd.read_csv('Data/rt.reviews.tsv', delimiter='\t', encoding='latin-1')
display(rt_reviews.head(2), len(rt_reviews))

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"


54432

## cleaning IMDB data files

In [10]:
# IMDB informantion about movie personel 
name = pd.read_csv('Data/imdb.name.basics.csv')
display(name.head(3), len(name))

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"


606648

In [11]:
#print(name.isna().sum()) # birth_year(523912), death_year(599865) Null so dropping both,
#primary_profession(51340), known_for_titles(30204) will keep because maybe importat to 
# always can scrape for it later 
name = name.drop(['birth_year', 'death_year'], axis = 1)

In [12]:
# IMDB informantion about titles 
takas = pd.read_csv('Data/imdb.title.akas.csv')
takas.rename(columns = {'title_id':'imdb_id'}, inplace = True)
display(takas.head(3), len(takas))

,imdb_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,NaN,imdbDisplay,NaN,0.0


331703

In [13]:
# checking data reducdencies
rand_id=np.random.randint(0,len(takas))
title=takas.loc[rand_id,'imdb_id']
display(takas.loc[(takas.imdb_id == title)])
'''checked with several random imdb_id's but all had totally convoluted data. the titles for the 
same imdb_id is completely different in most cases. This imdb.title.akas.csv data file is 
unsalvageable. file should not be used for further analysis.'''

,imdb_id,ordering,title,region,language,types,attributes,is_original_title
329213,tt3381790,1,"Norooz, Somewhere Else",NL,NaN,NaN,NaN,0.0


"checked with several random imdb_id's but all had totally convoluted data. the titles for the \nsame imdb_id is completely different in most cases. This imdb.title.akas.csv data file is \nunsalvageable. file should not be used for further analysis."

In [14]:
# IMDB datafile of title information could be important 
tbasics = pd.read_csv('Data/imdb.title.basics.csv')
tbasics.rename(columns = {'tconst':'imdb_id'}, inplace = True)
tbasics.rename(columns = {'start_year':'year'}, inplace = True)
display(tbasics.head(2),len(tbasics))

,imdb_id,primary_title,original_title,year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


146144

In [15]:
# check if the imdb_id is consistant here
rand_id=np.random.randint(0,len(tbasics))   # this is a really good way to randamly sample the file 
# tbasics.sample(3) # of course this is the standard way to do this. Wish I knew that before
title=tbasics.loc[rand_id,'imdb_id']
tbasics.loc[(tbasics.imdb_id == title) ]
# checked many imdb_id's and data is good and consistant. need some cleaning 

,imdb_id,primary_title,original_title,year,runtime_minutes,genres
72769,tt4152018,Lets Block,Nekolik let,2014,47.0,Documentary


In [ ]:
# clean runtime_minutes(31739), genres(5408). I wouldn't drop any null lines 
# as primary title is all non-null
# no cleaning needed
tbasics['primary_title']=tbasics.primary_title.map(lambda x: x.strip(), na_action='ignore')
tbasics['original_title']=tbasics.original_title.map(lambda x: x.strip(), na_action='ignore')
tbasics['original_title'].fillna(value='unknown',inplace=True)
tbasics['year'] = pd.to_numeric(tbasics['year'],errors='coerce')
tbasics['year'].fillna(value=3001 ,inplace=True)
tbasics['runtime_minutes'] = pd.to_numeric(tbasics['runtime_minutes'],errors='coerce')
tbasics['runtime_minutes'].fillna(value=-1.0 ,inplace=True)
tbasics['genres']=tbasics.genres.map(lambda x: x.strip(), na_action='ignore')
tbasics['genres'].fillna(value='unknown',inplace=True)
#tbasics.isna().sum() # it's clean
# save the cleaned file 
#tbasics.to_csv('Data/imdb.title.basics_clean.csv', index=False)

In [16]:
# IMDB all crew information per title needs imdb.name.basics.csv file 
crew = pd.read_csv('Data/imdb.title.crew.csv')
crew.rename(columns = {'tconst':'imdb_id'}, inplace = True)
display(crew.head(2),len(crew))

,imdb_id,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"


146144

In [17]:
# check if the imdb_id is consistant here
rand_id=np.random.randint(0,len(crew))   
title=crew.loc[rand_id,'imdb_id']
crew.loc[(crew.imdb_id == title) ]

,imdb_id,directors,writers
115843,tt7507124,nm6949661,NaN


In [18]:
#print("before\n", crew.isna().sum())
crew['directors']=crew.directors.map(lambda x: x.strip(), na_action='ignore')
crew['writers']=crew.writers.map(lambda x: x.strip(), na_action='ignore')
crew['writers'].fillna(value='unknown',inplace=True)
crew = crew[crew.writers != 'unknown']
crew['directors'].fillna(value='unknown',inplace=True)
crew = crew[crew.directors != 'unknown']
#print("after\n", crew.isna().sum())
# save the cleaned file 
#tbasics.to_csv('Data/imdb.title.crew_clean.csv', index=False)

In [19]:
# IMDB ratings 
ratings = pd.read_csv('Data/imdb.title.ratings.csv')
ratings.rename(columns = {'tconst':'imdb_id'}, inplace = True)
display(ratings.head(5), len(ratings))

,imdb_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


73856

In [29]:
title=['tt10356526', 'tt10384606', 'tt1042974', 'tt1043726', 'tt1060240']
for t in title:
    display(tbasics[tbasics.imdb_id == t ])

,imdb_id,primary_title,original_title,year,runtime_minutes,genres
3667,tt10356526,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance


,imdb_id,primary_title,original_title,year,runtime_minutes,genres
4103,tt10384606,Borderless,Borderless,2019,87.0,Documentary


,imdb_id,primary_title,original_title,year,runtime_minutes,genres
4721,tt1042974,Just Inès,Just Inès,2010,90.0,Drama


,imdb_id,primary_title,original_title,year,runtime_minutes,genres
4825,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy"


,imdb_id,primary_title,original_title,year,runtime_minutes,genres
5471,tt1060240,Até Onde?,Até Onde?,2011,73.0,"Mystery,Thriller"


In [ ]:
# check if the imdb_id is consistant here
rand_id=np.random.randint(0,len(ratings))   
title=ratings.loc[rand_id,'imdb_id']
ratings.loc[(ratings.imdb_id == title) ]

In [ ]:
#print("before:\n", ratings.isna().sum())
ratings['averagerating'] = pd.to_numeric(ratings['averagerating'],errors='coerce')
ratings['numvotes'] = pd.to_numeric(ratings['numvotes'],errors='coerce')
#print("after:\n", ratings.isna().sum()) # looks pretty clean
ratings.to_csv('Data/imdb.title.ratings_clean.csv', index=False)

In [ ]:
# this data maybe redundent 
principals = pd.read_csv('Data/imdb.title.principals.csv')
principals.rename(columns = {'tconst':'imdb_id'}, inplace = True)
display(principals.head(2),len(principals))

In [ ]:
#print("before:\n", principals.isna().sum())
principals.dropna(inplace=True) # null after cleaning. data is useless 
#print("after:\n", principals.isna().sum()) # looks pretty clean
#display(principals.head(2),len(principals))

## TheMovieDB data file

In [ ]:
# TheMovieDB.org
tmdb = pd.read_csv('Data/tmdb.movies.csv')

# we'll drop ['Unnamed: 0', 'genre_ids', 'popularity']
tmdb.drop(columns=['Unnamed: 0', 'genre_ids', 'popularity'], inplace=True) 
display(tmdb.head(3), len(tmdb))

In [ ]:
# comprihencive data in TMDB except IMDB_id
tmdb['original_language']=tmdb.original_language.map(lambda x: x.strip() , na_action='ignore')
tmdb['original_title']=tmdb.original_title.map(lambda x: x.strip() , na_action='ignore')
tmdb['release_date']=tmdb.release_date.map(lambda x: x.strip() , na_action='ignore')
tmdb['title']=tmdb.title.map(lambda x: x.strip() , na_action='ignore')

#print("before:\n", tmdb.isna().sum())
tmdb.dropna(inplace=True) # null after cleaning. data is useless 
#print("after:\n", tmdb.isna().sum()) # looks pretty clean now

# Alternative data from IMDBpro
credit: Jesse Newman for web scraping

In [ ]:
revenue=pd.read_csv('Data/revenue.csv')
# credit: Jesse Newman for web scraping
#renamed file of given data
date=pd.read_csv('Data/date.csv')
display(revenue.head(2),len(revenue))
display(date.head(2),len(date))

In [ ]:
# clean region_code
revenue['region_code'] = revenue['region_code'].str.replace('\[', '')
revenue['region_code'] = revenue['region_code'].str.replace('\]', '')
revenue.drop('rank', axis = 1, inplace=True)
revenue['imdb_id'].fillna(value='unknown',inplace=True)
revenue = revenue[revenue.imdb_id != 'unknown']
revenue['genres'].fillna(value='unknown',inplace=True)
revenue['director'].fillna(value='unknown',inplace=True)
revenue['region_code'].fillna(value='unknown',inplace=True)
#display(revenue.head(2))

In [ ]:
display(revenue.info())
display(date.info())

## Movie budget and gross revenue 

In [ ]:
# budget_usd   us_gross
#display(revenue.loc[revenue.budget_usd == -1]['budget_usd'].count())
display(revenue.loc[(revenue.budget_usd == -1) & (revenue.us_gross > 100000000)])
display(revenue.budget_usd.describe())
display(revenue.loc[revenue.us_gross < 0]['us_gross'].count())
display(revenue.us_gross.describe())

To understand the profitability of the movie both gross revenue and budget is needed. In this dataset with 14,431 entries, 6,534 entries do not have budget information. However, movies like <b>Wild Hogs</b> (\$ 168 million) and <b>Bambi</b> (\$ 102 million) clearly had boxoffice success. Thus, eliminating those data at this stage might not be predent. 

## Join date, runtime and rating to  revenue DataFrame

In [ ]:
# joining date to the revenue table 
date.drop(['title'], axis = 1, inplace=True)
date.sort_values(by=['imdb_id'], inplace=False)
revenue.sort_values(by=['imdb_id'], inplace=False)
date.set_index('imdb_id', inplace=True)
revenue.set_index('imdb_id', inplace=True)
revenue = revenue.join(date, how='left', lsuffix='1', rsuffix='1',  sort=True)
date.reset_index('imdb_id', inplace=True)
revenue.reset_index('imdb_id', inplace=True)
# clean date
#revenue['date'] = revenue['date'].astype('datetime64[ns]')
display(revenue.head(3))

In [ ]:
# joining runtime to the revenue table 
tbasics.drop(['primary_title', 'original_title', 'year', 'genres'], axis = 1, inplace=True)
tbasics.sort_values(by=['imdb_id'], inplace=False)
revenue.sort_values(by=['imdb_id'], inplace=False)
tbasics.set_index('imdb_id', inplace=True)
revenue.set_index('imdb_id', inplace=True)
revenue = revenue.join(tbasics, how='left', lsuffix='1', rsuffix='1',  sort=True)
tbasics.reset_index('imdb_id', inplace=True)
revenue.reset_index('imdb_id', inplace=True)
display(revenue.head(3))

In [ ]:
# joining ratings to the revenue table 
ratings.drop(['numvotes'], axis = 1, inplace=True)
ratings.sort_values(by=['imdb_id'], inplace=False)
revenue.sort_values(by=['imdb_id'], inplace=False)
ratings.set_index('imdb_id', inplace=True)
revenue.set_index('imdb_id', inplace=True)
revenue = revenue.join(ratings, how='left', lsuffix='1', rsuffix='1',  sort=True)
ratings.reset_index('imdb_id', inplace=True)
revenue.reset_index('imdb_id', inplace=True)
display(revenue.head(3))

In [ ]:
revenue.to_csv('Data/movie_main.csv', index=False)

In [ ]:
revenue.head(10)